<a href="https://colab.research.google.com/github/Mogaveera/atmanomalydetection/blob/master/atm_anomaly_4_classes_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing import image
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Flatten, ZeroPadding2D, Reshape, concatenate, BatchNormalization, Activation, Add
from keras.layers.convolutional import (Conv2D, MaxPooling2D, AveragePooling2D, Conv3D, MaxPooling3D)
from keras.layers import Bidirectional, TimeDistributed
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix
from keras.utils import np_utils

print(tf.__version__)

SEQ_LEN = 30
BATCH_SIZE = 16
EPOCHS = 100

train_video_index = []
test_video_index = []

1.15.0


Using TensorFlow backend.


In [0]:
!git clone https://github.com/Mogaveera/atmanomaly.git

Cloning into 'atmanomaly'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 203.93 MiB | 48.29 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (151/151), done.


In [0]:
def main():
    # Getting the data
    df = get_data('atmanomaly/Data/data_file.csv')
    
    # Clean the data
    #df_clean = clean_data(df)
    
    # Creating index-label maps and inverse_maps
    label_index, index_label = get_class_dict(df)
    
    # Split the dataset into train and test
    train, test = split_train_test(df)
    
    # Encoding the dataset
    train_video_index = make_dataset(train, label_index, "train")
    test_video_index = make_dataset(test, label_index,"test")
    return (train_video_index, test_video_index)

In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name']
    df = pd.read_csv(path,names=names)
    return df

def get_class_dict(df):
    class_name =  list(df['class'].unique())
    index = np.arange(0, len(class_name))
    label_index = dict(zip(class_name, index))
    index_label = dict(zip(index, class_name))
    return (label_index, index_label)
    
#def clean_data(df):
#    mask = np.logical_and(df['frames'] >= SEQ_LEN, df['frames'] <= MAX_SEQ_LEN)
#    df = df[mask]`
#    return df

def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (299,299))
    return preprocess_input(img)
    
    
def video_to_frame(row, label_index, phase, not_created):
    input_f = []
    output_y = []
    index = 0
    cap = cv2.VideoCapture(os.path.join("atmanomaly/Data","anomaly_dataset",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".mp4")) 
    #print(str(row["class"].iloc[0]))
    #print(str(row["video_name"].iloc[0]))
    frameno = 1
    imgs = []
    length = 0
    seq = 12
    while (cap.isOpened()):
      ret, frame = cap.read()
      if ret:
        if length < seq:
          if frameno % 10 == 0:
            frameno = frameno + 1
            frame = preprocess_image(frame)
            frame = image.img_to_array(frame)
            frame = frame / 255
            imgs.append(frame)
            length = length + 1
          else:
            frameno = frameno + 1
        else:
          seq = seq + 12
          imgs1 = np.array(imgs)
          input_f.append(imgs1)
          output_y.append(label_index)
          del imgs[:]
      else:
        break

    if not_created:
      f = h5py.File(phase+'_4'+'.h5', 'w')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        f.create_dataset(phase, data=input_f1, maxshape=(None, 12, 299, 299, 3))
        f.create_dataset(phase+"_labels", data=output_y1, maxshape=(None, 4))
        f.close()
    else:
      hf = h5py.File(phase+'_4'+'.h5', 'a')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        hf[phase].resize((hf[phase].shape[0] + input_f1.shape[0]), axis = 0)
        hf[phase][-input_f1.shape[0]:] = input_f1

        hf[phase+"_labels"].resize((hf[phase+"_labels"].shape[0] + output_y1.shape[0]), axis = 0)
        hf[phase+"_labels"][-output_y1.shape[0]:] = output_y1
        hf.close()

    del input_f[:]
    del output_y[:]
    del imgs[:]
    cap.release()
    return index
    
    


def make_dataset(data, label_index, phase):
    video_index = [0]
    required_classes = ["Arson", "Burglary", "Fighting", "normal"]
   
    not_created = True
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
            index = required_classes.index(str(data.iloc[[i]]["class"].iloc[0]))
            label_index = np.zeros((4))
            label_index[index] = 1
            index = video_to_frame(data.iloc[[i]], label_index, phase, not_created)
            real_index = video_index[-1] + index
            video_index.append(real_index)
            if real_index > 0:
              not_created = False

    return video_index

In [0]:
train_video_index, test_video_index = main()

100%|██████████| 24/24 [00:55<00:00,  3.01s/it]


In [0]:
x_train = HDF5Matrix('train_4.h5', 'train')
y_train = HDF5Matrix('train_4.h5', 'train_labels')
x_test = HDF5Matrix('test_4.h5', 'test')
y_test = HDF5Matrix('test_4.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[210])
print(x_test.shape)
print(y_test.shape)
print(y_test[161])
print(train_video_index)
print(test_video_index)

(702, 12, 299, 299, 3)
(702, 4)
[0. 1. 0. 0.]
(176, 12, 299, 299, 3)
(176, 4)
[0. 0. 0. 1.]
[0, 3, 6, 10, 13, 16, 20, 21, 23, 26, 29, 32, 37, 42, 46, 49, 57, 61, 64, 68, 68, 73, 78, 83, 86, 90, 94, 96, 99, 101, 104, 105, 106, 107, 112, 114, 118, 123, 124, 128, 131, 132, 134, 136, 145, 150, 156, 160, 164, 167, 171, 174, 180, 187, 192, 198, 217, 221, 244, 266, 269, 271, 278, 280, 282, 285, 286, 289, 295, 296, 298, 301, 312, 316, 327, 333, 334, 345, 363, 369, 378, 381, 387, 390, 398, 412, 417, 424, 425, 429, 434, 440, 445, 448, 457, 461, 470, 477, 485, 492, 499, 504, 514, 521, 529, 545, 550, 558, 565, 569, 578, 585, 592, 600, 610, 628, 641, 648, 656, 666, 672, 681, 685, 693, 696, 702]
[0, 1, 3, 4, 5, 9, 12, 17, 21, 42, 48, 60, 61, 68, 75, 87, 92, 110, 116, 134, 140, 150, 157, 163, 176]


In [0]:
def Inception(x, filters, name):

    f1, f2, f3 = filters

    one_filter = Conv3D(f1, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same', activation='relu',name=name+'_a')(x)

    one_three_filter = Conv3D(f2, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same', activation='relu',name=name+'_1b')(x)
    three_filter = Conv3D(f2, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu',name=name+'_b')(one_three_filter)

    one_five_filter = Conv3D(f3, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same', activation='relu',name=name+'_1c')(x)
    five_filter = Conv3D(f3, kernel_size=(5, 5, 5), strides=(1, 1, 1), padding='same', activation='relu',name=name+'_c')(one_five_filter)

    ouput = concatenate([five_filter, three_filter, one_filter], name=name+'_final')

    return ouput

In [0]:
def convolutional_block(X, f, filters, stage, s ):
    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv3D(F1, kernel_size=(1, 1, 1), strides=(1, s, s), padding='valid', name='conv'+stage+'_a')(X)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+ '_2a')(X)
    X = Activation('relu')(X)

    X = Conv3D(F2, kernel_size=(f, f, f), strides=(1, 1, 1), padding='same', name='conv'+stage+'_b')(X)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+ '_2b')(X)
    X = Activation('relu')(X)

    X = Conv3D(F3, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='valid', name='conv'+stage+'_c')(X)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+ '_2c')(X)

    X_shortcut = Conv3D(F3, kernel_size=(1, 1, 1), strides=(1, s, s), padding='valid', name='conv'+stage+'_main')(X_shortcut)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+ '_main')(X)

    X = Add()([X_shortcut, X])
    X = Activation('relu', name=stage+'_main')(X)

    return X

In [0]:
def identity_block(X, f, filters, stage, block):
    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv3D(F1, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='valid', name='conv'+stage+block+'_a')(X)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+block+'2a')(X)
    X = Activation('relu')(X)

    X = Conv3D(F2, kernel_size=(f, f, f), strides=(1, 1, 1), padding='same', name='conv'+stage+block+'_b')(X)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+block+'2b')(X)
    X = Activation('relu')(X)

    X = Conv3D(F3, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='valid', name='conv'+stage+block+'_c')(X)
    X = BatchNormalization(axis = 3, name = 'batch'+stage+block+'2c')(X)

    X = Add()([X_shortcut, X])
    X = Activation('relu', name=stage+block+'_main')(X)

    return X


In [0]:
def model():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (12, 299, 299, 3)

    main_input = Input(shape=input_shape, dtype='float32', name='main_input')

    x = Conv3D(32, kernel_size=(1, 3, 3), strides=(1, 1, 1), padding='valid', activation='relu',name='conv_1')(main_input)
    x = Conv3D(32, kernel_size=(1, 3, 3), strides=(1, 1, 1), padding='valid', activation='relu',name='conv_2')(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool_1')(x)


    x = Inception(x, [8, 24, 32], '1_Incept')
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool_2')(x)

    x = Inception(x, [8, 24, 32], '2_Incept')
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool_3')(x)

    # x = Inception(x, [16, 48, 64], '3_Incept')

    x = convolutional_block(x, 3, [32, 32, 128], '3', 1)
    x = identity_block(x, 3, [32, 32, 128], '3', '_1')
    x = identity_block(x, 3, [32, 32, 128], '3', '_2')
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool4')(x)

    x = convolutional_block(x, 3, [64, 64, 256], '4', 1)
    x = identity_block(x, 3, [64, 64, 256], '4', '_1')
    x = identity_block(x, 3, [64, 64, 256], '4', '_2')
    x = identity_block(x, 3, [64, 64, 256], '4', '_3')

    x = convolutional_block(x, 3, [128, 128, 512], '5', 1)
    x = identity_block(x, 3, [128, 128, 512], '5', '_1')
    x = identity_block(x, 3, [128, 128, 512], '5', '_2')
    x = identity_block(x, 3, [128, 128, 512], '5', '_3')

    x = MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 3, 3), padding='valid', name='pool_5')(x)

    # x = Inception(x, [64, 192, 256], '3_Incept')
    x = convolutional_block(x, 3, [128, 128, 512], '6', 1)
    x = identity_block(x, 3, [128, 128, 512], '6', '_1')
    x = identity_block(x, 3, [128, 128, 512], '6', '_2')
    x = identity_block(x, 3, [128, 128, 512], '6', '_3')
    x = MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 3, 3), padding='valid', name='pool_6')(x)

    x = Reshape((12, 2 * 2 * 512), input_shape=(12, 2, 2, 512))(x)
    
    x = Bidirectional(LSTM(2048))(x)
    x = Dropout(0.5)(x)
    
    x = Dense(512, activation='relu', name='fc9')(x)
    x = Dropout(0.5)(x)

    output = Dense(4, activation='softmax', name='main_output')(x)

    model = Model(inputs=main_input, outputs=output)


    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    callback_list = [checkpoint]
    
    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    #return model, callback_list
    #model.compile(optimizer = tf.train.AdamOptimizer(),
    #          loss = 'categorical_crossentropy',
    #        metrics=['accuracy'])
    return model, callback_list


In [0]:
model, callback_list = model()
model.summary(line_length=150)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
main_input (InputLayer)                          (None, 12, 299, 299, 3)          0                                                                   
______________________________________________________________________________________________________________________________________________________
conv_1 (Conv3D)                                  (None, 12, 297, 297, 32)         896               main_input[0][0]                                  
___________________________________________________________________________________________________________

In [0]:
model.fit(x_train, y_train, batch_size = 12, epochs = 10,verbose = 2, validation_data = (x_test, y_test), shuffle = 'batch',callbacks=callback_list)
model.save("atm_anomaly_4classes_usingconv3d2.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 702 samples, validate on 176 samples
Epoch 1/10





 - 233s - loss: 1.7145 - acc: 0.2849 - val_loss: 1.2794 - val_acc: 0.3409
Epoch 2/10
 - 312s - loss: 1.5370 - acc: 0.3390 - val_loss: 1.2795 - val_acc: 0.3409
Epoch 3/10
 - 150s - loss: 1.4264 - acc: 0.3575 - val_loss: 1.3083 - val_acc: 0.2784
Epoch 4/10
 - 142s - loss: 1.2139 - acc: 0.4402 - val_loss: 1.3119 - val_acc: 0.4148
Epoch 5/10
 - 130s - loss: 1.1007 - acc: 0.5128 - val_loss: 1.3193 - val_acc: 0.4432
Epoch 6/10
 - 130s - loss: 0.9396 - acc: 0.5769 - val_loss: 1.4366 - val_acc: 0.5170
Epoch 7/10
 - 129s - loss: 0.9524 - acc: 0.5712 - val_loss: 1.3327 - val_acc: 0.5455
Epoch 8/10
 - 129s - loss: 0.8799 - acc: 0.6182 - val_loss: 1.5027 - val_acc: 0.4375
Epoch 9/10
 - 128s - loss: 0.8093 - acc: 0.6239 - val_loss: 1.5495 - val_acc: 0.5852
Epoch 10/10
 - 128s - loss: 0.8928 - acc: 0.6011 - val_loss: 1.5523 - val_acc: 0.3977


In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=12)
print("accuracy: ", test_acc)

176/176 [==============================] - 12s 69ms/step
accuracy:  0.39772727238861
